In [30]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns


import jd

from sklearn.preprocessing import OrdinalEncoder,LabelEncoder,OneHotEncoder,TargetEncoder
from category_encoders import CountEncoder

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from imblearn.over_sampling import SMOTE, BorderlineSMOTE

from sklearn.impute import KNNImputer

from feature_engine.encoding import OrdinalEncoder as feat_eng_OrdinalCategoricalEncoder

import warnings
warnings.filterwarnings("ignore")

import pycaret
from pycaret.classification import *

In [ ]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df.status_group = le.fit_transform(df.status_group)


In [ ]:
for i,j in df.dtypes.items():
    if j == "object":
        df[i] = df[i].astype("category")

In [ ]:
df.dtypes

id                          int64
amount_tsh                float64
date_recorded            category
funder                   category
gps_height                  int64
installer                category
longitude                 float64
latitude                  float64
wpt_name                 category
num_private                 int64
basin                    category
subvillage               category
region                   category
region_code                 int64
district_code               int64
lga                      category
ward                     category
population                  int64
public_meeting           category
recorded_by              category
scheme_management        category
scheme_name              category
permit                   category
construction_year           int64
extraction_type          category
extraction_type_group    category
extraction_type_class    category
management               category
management_group         category
payment       

num_cols = df.select_dtypes(include = np.number).columns.tolist()
num_cols.remove("id")
num_cols.remove("status_group")

cat_cols= df.select_dtypes(exclude = np.number).columns.tolist()
cat_cols.remove("date_recorded")
cat_cols.remove("funder")
cat_cols.remove("installer")
cat_cols.remove("wpt_name")
cat_cols.remove("basin")


exp = setup(data=df, target='status_group', 
            numeric_features=num_cols, 
           categorical_features=["waterpoint_type_group","date_recorded","installer","wpt_name","basin","subvillage","region",
                                 "lga","ward",#"public_meeting","recorded_by","scheme_management","scheme_name"
                                 ],
            ignore_features=['id'])

best = exp.compare_models()

exp.plot_model(best,"confusion_matrix")

pred = exp.predict_model(best,X_test)
X_test_labels_pycaret = pred[["id","prediction_label"]]
X_test_labels_pycaret["prediction_label"] = le.inverse_transform(X_test_labels_pycaret.prediction_label)
X_test_labels_pycaret.rename(columns={"prediction_label":"status_group"},inplace=True)
X_test_labels_pycaret.to_csv("Out/predicciones_pycaret1.csv",index=None,sep=",")

In [ ]:
pd.read_csv("Out/predicciones_pycaret1.csv")

,id,status_group
0,50785,functional
1,51630,non functional
2,17168,non functional
3,45559,non functional
4,49871,functional
...,...,...
14845,39307,functional
14846,18990,functional
14847,28749,functional
14848,33492,functional


In [3]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")



In [4]:
for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#df.status_group = le.fit_transform(df.status_group)

In [6]:
y["status_group"] = le.fit_transform(y.status_group)

# drop list baja V de Cramer para que no estorben, posteriormente cambio filosofía.
drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]
X.drop(drop_list,axis=1,inplace=True)
X_test.drop(drop_list,axis=1,inplace=True)

cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()


cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
cat_cols.append("district_code")
cat_cols.append("region_code")

num_cols = X.select_dtypes(include=np.number).columns.tolist()
num_cols.remove("id")
num_cols.remove("district_code")
num_cols.remove("region_code")

preprocessing = ColumnTransformer([
    ('target_enc', TargetEncoder(), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)
X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())

In [7]:
rf = RandomForestClassifier()
rf.fit(X.to_numpy(),y.status_group.values)

RandomForestClassifier()

In [8]:
X_test["predictions"] = rf.predict(X_test)

In [9]:
X_test.predictions = le.inverse_transform(X_test.predictions)
X_test = X_test[["remainder__id","predictions"]]
X_test.remainder__id = X_test.remainder__id.astype(int)

X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)
X_test.to_csv("Out/predicciones_TargetEncoderStandardScaler.csv",index=None,sep=",")

In [10]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")

for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")


In [11]:
y["status_group"] = le.fit_transform(y.status_group)

drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]
X.drop(drop_list,axis=1,inplace=True)
X_test.drop(drop_list,axis=1,inplace=True)

cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()


cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
cat_cols.append("district_code")
cat_cols.append("region_code")

num_cols = X.select_dtypes(include=np.number).columns.tolist()
num_cols.remove("id")
num_cols.remove("district_code")
num_cols.remove("region_code")

preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =10000000), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)
X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())

In [12]:
X.isna().sum()

target_enc__date_recorded            0
target_enc__funder                   0
target_enc__installer                0
target_enc__wpt_name                 0
target_enc__basin                    0
target_enc__subvillage               0
target_enc__region                   0
target_enc__lga                      0
target_enc__ward                     0
target_enc__public_meeting           0
target_enc__scheme_management        0
target_enc__scheme_name              0
target_enc__extraction_type          0
target_enc__extraction_type_group    0
target_enc__extraction_type_class    0
target_enc__management               0
target_enc__payment                  0
target_enc__payment_type             0
target_enc__water_quality            0
target_enc__quality_group            0
target_enc__quantity                 0
target_enc__quantity_group           0
target_enc__source                   0
target_enc__source_type              0
target_enc__source_class             0
target_enc__waterpoint_ty

In [13]:
rf = RandomForestClassifier()
rf.fit(X.to_numpy(),y.status_group.values)
X_test["status_group"] = rf.predict(X_test)
X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

In [14]:
X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/predicciones_OrdinalEncoderStandardScaler.csv",index=None,sep=",")


# Con Clusters de location, tenía altas espectativas, pero no dio buen resultado

In [15]:
le = OrdinalEncoder()
label_enc = LabelEncoder()
X= pd.read_csv("Out/XReemplazoLongitudeLatitudeConClusters.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("Out/X_testReemplazoLongitudeLatitudeConClusters.csv")
df = pd.merge(X,y,on="id")

for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")
    
y["status_group"] = label_enc.fit_transform(y.status_group)

drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]
X.drop(drop_list,axis=1,inplace=True)
X_test.drop(drop_list,axis=1,inplace=True)

cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()


cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
cat_cols.append("district_code")
cat_cols.append("region_code")

num_cols = X.select_dtypes(include=np.number).columns.tolist()
num_cols.remove("id")
num_cols.remove("district_code")
num_cols.remove("region_code")

preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =10000000), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)
X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())

rf = RandomForestClassifier()
rf.fit(X.to_numpy(),y.status_group.values)
X_test["status_group"] = rf.predict(X_test)
X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)


In [16]:
X_test = X_test[["id","status_group"]]
X_test.status_group = label_enc.inverse_transform(X_test.status_group)
X_test.to_csv("Out/predicciones_OrdinalEncoderStandardScaler7ClustersLoc.csv",index=None,sep=",")

# GradientBoosting da buenos resultados en cross_validate, es extraño que el GradientBoosting le gane al random forest en cross validate, pero sea mucho peor en el test

In [17]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")

for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")


In [18]:
le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)

drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]
X.drop(drop_list,axis=1,inplace=True)
X_test.drop(drop_list,axis=1,inplace=True)

cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()


cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
cat_cols.append("district_code")
cat_cols.append("region_code")

num_cols = X.select_dtypes(include=np.number).columns.tolist()
num_cols.remove("id")
num_cols.remove("district_code")
num_cols.remove("region_code")

preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =10000000), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)
X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())

In [19]:
gradBoost = GradientBoostingClassifier()
gradBoost.fit(X.to_numpy(),y.status_group.values)
X_test["status_group"] = gradBoost.predict(X_test)
X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

In [20]:
X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/GradBoostpredicciones_OrdinalEncoderStandardScaler.csv",index=None,sep=",")

# Empiezo a probra resampling, todos empeoran el score

## SMOTE not majority

In [21]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")
sm = SMOTE(random_state=23,sampling_strategy = "not majority")

for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)

drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]
X.drop(drop_list,axis=1,inplace=True)
X_test.drop(drop_list,axis=1,inplace=True)

cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()


cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
cat_cols.append("district_code")
cat_cols.append("region_code")

num_cols = X.select_dtypes(include=np.number).columns.tolist()
num_cols.remove("id")
num_cols.remove("district_code")
num_cols.remove("region_code")

preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =10000000), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)
X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())

X, y = sm.fit_resample(X, y.status_group.values)

rf = RandomForestClassifier()
rf.fit(X.to_numpy(),y)
X_test["status_group"] = rf.predict(X_test)
X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

In [22]:
X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/RandomForestSMOTEnot_majority.csv",index=None,sep=",")

## SMOTE minority

In [23]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")
sm = SMOTE(random_state=23,sampling_strategy = "minority")

for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)

drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]
X.drop(drop_list,axis=1,inplace=True)
X_test.drop(drop_list,axis=1,inplace=True)

cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()


cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
cat_cols.append("district_code")
cat_cols.append("region_code")

num_cols = X.select_dtypes(include=np.number).columns.tolist()
num_cols.remove("id")
num_cols.remove("district_code")
num_cols.remove("region_code")

preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =10000000), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)
X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())

X, y = sm.fit_resample(X, y.status_group.values)

rf = RandomForestClassifier()
rf.fit(X.to_numpy(),y)
X_test["status_group"] = rf.predict(X_test)
X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

In [24]:
X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/RandomForestSMOTEminority.csv",index=None,sep=",")

In [25]:
# queda por intentar con más técnicas, dejando clusters y continuas

## Quiero conocer la distribución de las etiquetas de prueba, para eso utilizaré 3 entregas con una sola etiqueta incluida en ellas

In [26]:
solo_functional = pd.read_csv("Out/RandomForestSMOTEminority.csv",sep=",")
solo_functional.status_group = "functional"
solo_functional.to_csv("Out/solo_functional.csv",index=None,sep=",")

In [27]:
pd.read_csv("Out/RandomForestSMOTEminority.csv",sep=",").status_group.unique()

array(['non functional', 'functional', 'functional needs repair'],
      dtype=object)

In [28]:
solo_nonfunctional = pd.read_csv("Out/RandomForestSMOTEminority.csv",sep=",")
solo_nonfunctional.status_group = "non functional"
solo_nonfunctional.to_csv("Out/solo_nonfunctional.csv",index=None,sep=",")


In [29]:
solo_functional_needs_repair = pd.read_csv("Out/RandomForestSMOTEminority.csv",sep=",")
solo_functional_needs_repair.status_group = "functional needs repair"
solo_functional_needs_repair.to_csv("Out/solo_functional_needs_repair.csv",index=None,sep=",")
solo_functional_needs_repair

,id,status_group
0,50785,functional needs repair
1,51630,functional needs repair
2,17168,functional needs repair
3,45559,functional needs repair
4,49871,functional needs repair
...,...,...
14845,39307,functional needs repair
14846,18990,functional needs repair
14847,28749,functional needs repair
14848,33492,functional needs repair


In [30]:
solo_functional = "0.5461"
solo_nonfunctional = "0.3820"
solo_functional_needs_repair = "0.0719"
0.5461+0.3820+0.0719

1.0

In [31]:
pd.read_csv("In/Training_set_labels.csv").status_group.value_counts(normalize=1)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

# Intento distintos encoders de missings

#### "https://scikit-learn.org/stable/auto_examples/impute/plot_iterative_imputer_variants_comparison.html#sphx-glr-auto-examples-impute-plot-iterative-imputer-variants-comparison-py"

In [32]:
drop_list  = ["scheme_name","funder","management_group","num_private","recorded_by",
              "extraction_type_group","extraction_type_class","payment","quality_group",
              "quantity_group","source","source_class","source_type","waterpoint_type_group"]

In [33]:
X = pd.read_csv("In/Training_set_values.csv")
X.drop(drop_list,axis=1,inplace=True)
for i in X.select_dtypes(include = "O").columns:
    X[i] = X[i].astype("category")
X.district_code = X.district_code.astype("category")
X.region_code = X.region_code.astype("category")

X_test = pd.read_csv("In/Test_set_values.csv")
X_test.drop(drop_list,axis=1,inplace=True)
for i in X_test.select_dtypes(include = "O").columns:
    X_test[i] = X[i].astype("category")
X_test.district_code = X_test.district_code.astype("category")
X_test.region_code = X_test.region_code.astype("category")


y = pd.read_csv("In/Training_set_labels.csv")
le = LabelEncoder()
y = le.fit_transform(y.status_group)

X.loc[X["amount_tsh"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
X.loc[X["amount_tsh"].between(1.5,np.inf),"amount_tsh_binned"] = 1

X_test.loc[X["amount_tsh"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
X_test.loc[X["amount_tsh"].between(1.5,np.inf),"amount_tsh_binned"] = 1

from sklearn.cluster import KMeans

clustering = KMeans(n_clusters=20)
#df_geo = df[["longitude","latitude"]]
#df_geo["location_cluster"] = clustering.fit_predict(df_geo)

X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())

### Primero debo ver cómo se comporta el score con los drops llevados a cabo

In [34]:
X.select_dtypes(include=np.number).columns.tolist()

['id',
 'amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'population',
 'construction_year',
 'amount_tsh_binned',
 'location_cluster']

In [35]:
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")


preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =10000000), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())

rf = RandomForestClassifier()
rf.fit(X.to_numpy(),y)
X_test["status_group"] = rf.predict(X_test)
X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

In [36]:
X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/MismoRFBuenoConMasDropsClustersYCont.csv",index=None,sep=",")

In [37]:
X_test

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,non functional
4,49871,functional
...,...,...
14845,39307,non functional
14846,18990,functional
14847,28749,functional
14848,33492,functional


# ¿Esto dio un score de 0.5????? 

## Tendré que volver a intentar pero sin drop list

In [38]:
drop_list =[]
X = pd.read_csv("In/Training_set_values.csv")
X.drop(drop_list,axis=1,inplace=True)
for i in X.select_dtypes(include = "O").columns:
    X[i] = X[i].astype("category")
X.district_code = X.district_code.astype("category")
X.region_code = X.region_code.astype("category")

X_test = pd.read_csv("In/Test_set_values.csv")
X_test.drop(drop_list,axis=1,inplace=True)
for i in X_test.select_dtypes(include = "O").columns:
    X_test[i] = X[i].astype("category")
X_test.district_code = X_test.district_code.astype("category")
X_test.region_code = X_test.region_code.astype("category")


y = pd.read_csv("In/Training_set_labels.csv")
le = LabelEncoder()
y = le.fit_transform(y.status_group)

X.loc[X["amount_tsh"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
X.loc[X["amount_tsh"].between(1.5,np.inf),"amount_tsh_binned"] = 0

X_test.loc[X["amount_tsh"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
X_test.loc[X["amount_tsh"].between(1.5,np.inf),"amount_tsh_binned"] = 0

from sklearn.cluster import KMeans

clustering = KMeans(n_clusters=20)
#df_geo = df[["longitude","latitude"]]
#df_geo["location_cluster"] = clustering.fit_predict(df_geo)

X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())

cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")


preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =10000000), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())

rf = RandomForestClassifier()
rf.fit(X.to_numpy(),y)
X_test["status_group"] = rf.predict(X_test)
X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/MismoRFSinDrops.csv",index=None,sep=",")

# Volvió a dar 0.5, no entiendo lo que ocurrió con este submission, probablemente un error de Jupyter de alguna variable ya creada en el entorno estorbando

# Intento ahora mismo código utilizado con SMOTE, pero quitando las dos líneas del resampling a ver si es que la página tiene un problema

### 0.8160

In [39]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")

for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)

drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]
X.drop(drop_list,axis=1,inplace=True)
X_test.drop(drop_list,axis=1,inplace=True)

cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()


cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
cat_cols.append("district_code")
cat_cols.append("region_code")

num_cols = X.select_dtypes(include=np.number).columns.tolist()
num_cols.remove("id")
num_cols.remove("district_code")
num_cols.remove("region_code")

preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =10000000), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
print("ya")
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())
print("ya")

rf = RandomForestClassifier()
rf.fit(X.to_numpy(),y.status_group.values)

X_test["status_group"] = rf.predict(X_test)
X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

ya
ya


In [40]:
X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/IntentoVolver08.csv",index=None,sep=",")

# Esto ya volvió a funcionar y a darme score 0.8, no entiendo bien lo que ocurre. Lo único que pudiese ser es que este código le hace drop a amount_tsh y el resto no

# Menos mal revisé los resultados del cambio a la lista de drop antes de intentar los imputadores, hubiese pensado que el problema venía de la imputación.

# Ahora sí intento los imputadores de missings en conjunto a un drop_list sacado con una limpieza de variables que posteriormente se borró por error e intenté reconstruir

#### "https://scikit-learn.org/stable/auto_examples/impute/plot_iterative_imputer_variants_comparison.html#sphx-glr-auto-examples-impute-plot-iterative-imputer-variants-comparison-py"

In [41]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.pipeline import make_pipeline
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import Ridge

from sklearn.impute import KNNImputer

# KNN imputer, 8170 MEJOR RESULTADO HASTA AHORA

In [42]:
it_imp2 = KNNImputer()

# Me imagino que: KNNImputer()==IterativeImputer(estimator=KNeighborsRegressor())

In [43]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")

for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)


drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]

X.drop(drop_list,axis=1,inplace=True)
X_test.drop(drop_list,axis=1,inplace=True)

cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

cat_cols.append("district_code")
cat_cols.append("region_code")

num_cols.remove("id")
num_cols.remove("district_code")
num_cols.remove("region_code")



preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


X = it_imp2.fit_transform(X)
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier()
rf.fit(X,y.status_group.values)
y_pred = rf.predict(X_test)

X_test["status_group"] = rf.predict(X_test)

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/KneighborsImputer.csv",index=None,sep=",")

# random forest imputer

In [44]:
it_imp1 = IterativeImputer(RandomForestRegressor(n_jobs=-1))

In [45]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")

for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)

drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]
X.drop(drop_list,axis=1,inplace=True)
X_test.drop(drop_list,axis=1,inplace=True)

cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()


cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
cat_cols.append("district_code")
cat_cols.append("region_code")

num_cols = X.select_dtypes(include=np.number).columns.tolist()
num_cols.remove("id")
num_cols.remove("district_code")
num_cols.remove("region_code")



preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())

X = it_imp1.fit_transform(X)
print("ya")
#X_test = it_imp1.transform(X)
X_test = pd.DataFrame(data=it_imp1.transform(X_test), columns=it_imp1.get_feature_names_out())
print("ya")

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X.to_numpy(),y.status_group.values)

X_test["status_group"] = rf.predict(X_test)
X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/RFImputer.csv",index=None,sep=",")
# 37 minutos y no hace ni el primer print. No vale la pena.

KeyboardInterrupt: 

# pipeline(Nystroem(),Ridge()) imputer
### 0.8153

In [ ]:
it_imp3 = IterativeImputer(make_pipeline(Nystroem(),Ridge()))

In [ ]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")

for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)

drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]
X.drop(drop_list,axis=1,inplace=True)
X_test.drop(drop_list,axis=1,inplace=True)

cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()


cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
cat_cols.append("district_code")
cat_cols.append("region_code")

num_cols = X.select_dtypes(include=np.number).columns.tolist()
num_cols.remove("id")
num_cols.remove("district_code")
num_cols.remove("region_code")


preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


X = it_imp3.fit_transform(X)
#X_test = it_imp3.transform(X)
X_test = pd.DataFrame(data=it_imp3.transform(X_test), columns=it_imp3.get_feature_names_out())

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X,y.status_group.values)

X_test["status_group"] = rf.predict(X_test)
X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/NystroemRidgeImputer.csv",index=None,sep=",")

c:\Users\jrios_bipicar\anaconda3\Lib\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


## Vuelvo a intentar cambio en el drop list, en este caso añado los drops de la lista pequeña que no estaban incluidos en la grande, esto lo hago porque la lista grande dió un resultado demasiado parecido a aquel logrado sin ningún drop.

#### Dio peor resultado que lista pequeña(0.8170), pero sólo un poco peor.

#### 0.8133

In [ ]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")

for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)

#drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]
drop_list  = ["scheme_name","funder","management_group","num_private","recorded_by",
              "extraction_type_group","extraction_type_class","payment","quality_group",
              "quantity_group","source","source_class","source_type","waterpoint_type_group"]
X.drop(drop_list,axis=1,inplace=True)
X_test.drop(drop_list,axis=1,inplace=True)

cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

cat_cols.append("district_code")
cat_cols.append("region_code")

num_cols.remove("id")
num_cols.remove("district_code")
num_cols.remove("region_code")



preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


X = it_imp2.fit_transform(X)
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier()
rf.fit(X,y.status_group.values)
y_pred = rf.predict(X_test)

X_test["status_group"] = rf.predict(X_test)

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/KneighborsImputer.csv",index=None,sep=",")

# KNN IMP CON NUEVOS DROPS Y TODOS LOS FEATURES INGENIADOS

# 0.850

In [ ]:
it_imp2 = it_imp2 = KNNImputer()

X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")


for i in X.select_dtypes(include = "O").columns:
    X[i] = X[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)


#fechas------------------------------------------------------------------------            

X.date_recorded = X.date_recorded.astype("O")
X.date_recorded = pd.to_datetime(X.date_recorded)
X['date_recorded'] = (X.date_recorded.max() - X.date_recorded).dt.days



X_test.date_recorded = X_test.date_recorded.astype("O")
X_test.date_recorded = pd.to_datetime(X_test.date_recorded)
X_test['date_recorded'] = (X_test.date_recorded.max() -X_test.date_recorded).dt.days # X.date_recorded.max() a propósito

#Droplist-----------------------------------------------------------------------------------------------------------------------------------------------   
drop_list  = ["management_group","num_private","recorded_by","region","region_code",
              "extraction_type_group","extraction_type_class","payment","quality_group",
              "quantity_group","source_class","source_type","waterpoint_type_group",
              ]#"permit","population","amount_tsh" eran drops que funcionaban bien al principio
X.drop(drop_list,axis=1,inplace=True)
X_test.drop(drop_list,axis=1,inplace=True)

#Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------   
X.loc[X["amount_tsh"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
X.loc[X["amount_tsh"].between(1.5,np.inf),"amount_tsh_binned"] = 1

X_test.loc[X["amount_tsh"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
X_test.loc[X["amount_tsh"].between(1.5,np.inf),"amount_tsh_binned"] = 1

#Clusters de latitud y longitud------------------------------------------------------------------------   
from sklearn.cluster import KMeans
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())
#-----------------------------------------------------------------------------------------------------------------------------------------------    
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist() 
cat_cols.append("district_code")
num_cols.remove("district_code")
num_cols.remove("id")



preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())



X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier()
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
#y_pred = rf.predict(X_test.drop("id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/NuevaDropListKNNimpTodasFeatures.csv",index=None,sep=",")

/Users/juandavid/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


## Intento haciendo los drops que hago al principo por baja V de Cramwe con objetivo:

## 0.8152

In [ ]:
it_imp2 = it_imp2 = KNNImputer()

X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")


for i in X.select_dtypes(include = "O").columns:
    X[i] = X[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)


#fechas------------------------------------------------------------------------            

X.date_recorded = X.date_recorded.astype("O")
X.date_recorded = pd.to_datetime(X.date_recorded)
X['date_recorded'] = (X.date_recorded.max() - X.date_recorded).dt.days



X_test.date_recorded = X_test.date_recorded.astype("O")
X_test.date_recorded = pd.to_datetime(X_test.date_recorded)
X_test['date_recorded'] = (X_test.date_recorded.max() -X_test.date_recorded).dt.days # X.date_recorded.max() a propósito

#Droplist-----------------------------------------------------------------------------------------------------------------------------------------------   
drop_list  = ["management_group","num_private","recorded_by","region","region_code",
              "extraction_type_group","extraction_type_class","payment","quality_group",
              "quantity_group","source_class","source_type","waterpoint_type_group",
              "permit","population","amount_tsh"] #permit","population","amount_tsh" eran drops que funcionaban bien al principio
X.drop(drop_list,axis=1,inplace=True)
X_test.drop(drop_list,axis=1,inplace=True)

#Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------   
# no se hace porque le hago drop al amount_tsh  

#Clusters de latitud y longitud------------------------------------------------------------------------   
from sklearn.cluster import KMeans
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())
#-----------------------------------------------------------------------------------------------------------------------------------------------    
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist() 
cat_cols.append("district_code")
num_cols.remove("district_code")
num_cols.remove("id")



preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())



X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier()
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
#y_pred = rf.predict(X_test.drop("id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/NuevaDropListMasDropsVCramerKNNimpTodasFeatures.csv",index=None,sep=",")

/Users/juandavid/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


# Sin mis features a ver si están sesgando los resultados

## 0.8119 --> algo bueno hacen mis features

In [ ]:
it_imp2 = KNNImputer()

X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")


for i in X.select_dtypes(include = "O").columns:
    X[i] = X[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)

#Droplist-----------------------------------------------------------------------------------------------------------------------------------------------   
drop_list  = ["management_group","num_private","recorded_by","region","region_code",
              "extraction_type_group","extraction_type_class","payment","quality_group",
              "quantity_group","source_class","source_type","waterpoint_type_group",
              "permit","population","amount_tsh"] #permit","population","amount_tsh" eran drops que funcionaban bien al principio
X.drop(drop_list,axis=1,inplace=True)
X_test.drop(drop_list,axis=1,inplace=True)
#-----------------------------------------------------------------------------------------------------------------------------------------------    
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist() 
cat_cols.append("district_code")
num_cols.remove("district_code")
num_cols.remove("id")



preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())



X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier()
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
#y_pred = rf.predict(X_test.drop("id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/NuevaDropListMasDropsVCramerKNNimpNingunFeature.csv",index=None,sep=",")

In [ ]:
## deberiái ahora usar la lista que me gusta sacando region code

## knn imputer que da los mejores resultados hasta ahora con mis features

## 0.8098

In [ ]:
it_imp2 = KNNImputer()

X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")

for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)
#fechas------------------------------------------------------------------------            

X.date_recorded = X.date_recorded.astype("O")
X.date_recorded = pd.to_datetime(X.date_recorded)
X['date_recorded'] = (X.date_recorded.max() - X.date_recorded).dt.days

X_test.date_recorded = X_test.date_recorded.astype("O")
X_test.date_recorded = pd.to_datetime(X_test.date_recorded)
X_test['date_recorded'] = (X_test.date_recorded.max() -X_test.date_recorded).dt.days # X.date_recorded.max() a propósito
#-----------------------------------------------

drop_list  = ["scheme_name","funder","management_group","num_private","recorded_by",
              "extraction_type_group","extraction_type_class","payment","quality_group",
              "quantity_group","source","source_class","source_type","waterpoint_type_group"]

X.drop(drop_list,axis=1,inplace=True)
X_test.drop(drop_list,axis=1,inplace=True)
#Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------   
X.loc[X["amount_tsh"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
X.loc[X["amount_tsh"].between(1.5,np.inf),"amount_tsh_binned"] = 1

X_test.loc[X["amount_tsh"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
X_test.loc[X["amount_tsh"].between(1.5,np.inf),"amount_tsh_binned"] = 1

#Clusters de latitud y longitud------------------------------------------------------------------------   
from sklearn.cluster import KMeans
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())
#------------------------------------------------------------------------  ------------------------------------------------------------------------  
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

cat_cols.append("district_code")
cat_cols.append("region_code")

num_cols.remove("id")
num_cols.remove("district_code")
num_cols.remove("region_code")



preprocessing = ColumnTransformer([
    ('target_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


X = it_imp2.fit_transform(X)
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier()
rf.fit(X,y.status_group.values)
y_pred = rf.predict(X_test)

X_test["status_group"] = rf.predict(X_test)

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/KneighborsImputerYMisFeatures.csv",index=None,sep=",")

## Con función que elige el mejor encoding

## drop list con sentido  
["management_group","num_private","recorded_by","region","region_code",
              "extraction_type_group","extraction_type_class","payment","quality_group",
              "quantity_group","source_class","source_type","waterpoint_type_group",
              "permit","population","amount_tsh"]

In [ ]:
def preprocess(X):
    for i in X.select_dtypes(include = "O").columns:
        X[i] = X[i].astype("category")

    #Fechas------------------------------------------------------------------------ 
    X.date_recorded = X.date_recorded.astype("O")
    X.date_recorded = pd.to_datetime(X.date_recorded)
    X['date_recorded'] = (X.date_recorded.max() - X.date_recorded).dt.days
    #Drop list ------------------------------------------------------------------------ 
    drop_list  = ["management_group","num_private","recorded_by","region","region_code",
              "extraction_type_group","extraction_type_class","payment","quality_group",
              "quantity_group","source_class","source_type","waterpoint_type_group",
              "permit","population"]
    #Clusters de latitud y longitud------------------------------------------------------------------------   
    X.drop(drop_list,axis=1,inplace=True)
    clustering = KMeans(n_clusters=20)
    X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
    
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------   
    X["amount_tsh_binned"] = X["amount_tsh"].copy()
    X.loc[X["amount_tsh_binned"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
    X.loc[X["amount_tsh_binned"].between(1.5,np.inf),"amount_tsh_binned"] = 1
    X["amount_tsh_binned"] = X["amount_tsh_binned"].astype("category")
    return X

## Con mi función en modo un solo test:
### 0.8143

In [ ]:
import sklearn
sklearn.set_config(transform_output="pandas")
it_imp2 = it_imp2 = KNNImputer()

X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")


le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)


X= preprocess(X)
X_test = preprocess(X_test)

OrdinalEncoder_cols = ['installer', 'basin', 'ward', 'quantity', 'source', 'amount_tsh_binned']

OnehotEnc_cols = ['wpt_name', 'subvillage', 'public_meeting', 'scheme_name', 'extraction_type', 'management']

TargetEnc_cols = ['water_quality']
FrequencyEnc_cols =['funder', 'district_code', 'lga', 'scheme_management', 'payment_type', 'waterpoint_type']


num_cols = X.select_dtypes(include=np.number).columns.tolist() 
num_cols.remove("district_code")
num_cols.remove("id")



preprocessing = ColumnTransformer([
        ('ordinal', OrdinalEncoder(handle_unknown="use_encoded_value",
                                    unknown_value=10000000,
                                    encoded_missing_value =10000001,
                                    min_frequency = 0.01), OrdinalEncoder_cols),
        #arreglar:
        ('onehot', OneHotEncoder(max_categories=7,# Si existen más de 6 valores únicos crea categoría "infrequent"
                                    handle_unknown ="infrequent_if_exist",
                                    drop="first",sparse_output=False), OnehotEnc_cols),

        ('target', TargetEncoder(target_type="multiclass",
                                    cv=2,), TargetEnc_cols),  # default cv 5 me tardaría una eternidad
                                    
        ('Frequency', CountEncoder(handle_unknown="value",
                                    handle_missing="value"), FrequencyEnc_cols),                             
        ('scaler', StandardScaler(), num_cols),  
    ],
    remainder = "passthrough",
    )

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
#y_pred = rf.predict(X_test.drop("id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/MiFuncionModoUnTest.csv",index=None,sep=",")

## Mi Función en modo KFold
### 0.8135

In [ ]:
import sklearn
sklearn.set_config(transform_output="pandas")
it_imp2 = it_imp2 = KNNImputer()

X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")


le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)


X= preprocess(X)
X_test = preprocess(X_test)

OrdinalEncoder_cols = ['funder', 'quantity', 'source', 'waterpoint_type']

OnehotEnc_cols = ['basin', 'district_code', 'ward', 'scheme_management', 'scheme_name', 'extraction_type',
                'management', 'payment_type', 'water_quality', 'amount_tsh_binned']

TargetEnc_cols = []
FrequencyEnc_cols =['installer', 'wpt_name', 'subvillage', 'lga', 'public_meeting']


num_cols = X.select_dtypes(include=np.number).columns.tolist() 
num_cols.remove("district_code")
num_cols.remove("id")



preprocessing = ColumnTransformer([
        ('ordinal', OrdinalEncoder(handle_unknown="use_encoded_value",
                                    unknown_value=10000000,
                                    encoded_missing_value =10000001,
                                    min_frequency = 0.01), OrdinalEncoder_cols),
        #arreglar:
        ('onehot', OneHotEncoder(max_categories=7,# Si existen más de 6 valores únicos crea categoría "infrequent"
                                    handle_unknown ="infrequent_if_exist",
                                    drop="first",sparse_output=False), OnehotEnc_cols),

        ('target', TargetEncoder(target_type="multiclass",
                                    cv=2,), TargetEnc_cols),  # default cv 5 me tardaría una eternidad
                                    
        ('Frequency', CountEncoder(handle_unknown="value",
                                    handle_missing="value"), FrequencyEnc_cols),                             
        ('scaler', StandardScaler(), num_cols),  
    ],
    remainder = "passthrough",
    )

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())

X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
#y_pred = rf.predict(X_test.drop("id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/MiFuncionModoFold.csv",index=None,sep=",")

## El rf + knnImputer + la drop_list original pero ahora con todas las features y arreglos

# EUREKA!!! 0.8174!! jajajaja

In [ ]:
def preprocess(X):
    for i in X.select_dtypes(include = "O").columns:
        X[i] = X[i].astype("category")

    #Fechas------------------------------------------------------------------------ 
    X.date_recorded = X.date_recorded.astype("O")
    X.date_recorded = pd.to_datetime(X.date_recorded)
    X['date_recorded'] = (X.date_recorded.max() - X.date_recorded).dt.days
    X.district_code = X.district_code.astype("category")
    X.region_code = X.region_code.astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------   
    X["amount_tsh_binned"] = X["amount_tsh"].copy()
    X.loc[X["amount_tsh_binned"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
    X.loc[X["amount_tsh_binned"].between(1.5,np.inf),"amount_tsh_binned"] = 1
    X["amount_tsh_binned"] = X["amount_tsh_binned"].astype("category")
    #gps_heightoptbinning------------------------------------------------------------------------  
    X.loc[X["gps_height"]<0,"gps_height"] = 0
    X["gps_height_binned"] = X["gps_height"].copy()
    X.loc[X["gps_height"].between(-np.inf,508),"gps_height_binned"] = "bin0"
    X.loc[X["gps_height"].between(508,850.5),"gps_height_binned"] = "bin1"
    X.loc[X["gps_height"].between(850.5,1359.5),"gps_height_binned"] = "bin2"
    X.loc[X["gps_height"].between(1359.5,1567.50),"gps_height_binned"] = "bin3"
    X.loc[X["gps_height"].between(1567.50,1688.50),"gps_height_binned"] = "bin4"
    X.loc[X["gps_height"].between(1688.50,np.inf),"gps_height_binned"] = "bin5"
    X["gps_height_binned"] = X["gps_height_binned"].astype("category")
    #population_binned optbinning------------------------------------------------------------------------ 
    X["population_binned"] = X["population"].copy()
    X.loc[X["population"].between(-np.inf,1.5),"population_binned"] = "bin0"
    X.loc[X["population"].between(1.5,np.inf),"population_binned"] = "bin1"
    X["population_binned"] = X["population_binned"].astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------  
    X["date_recorded_binned"] = X["date_recorded"].copy()
    X.loc[X["date_recorded"].between(-np.inf,874.50),"date_recorded_binned"] = "bin0"
    X.loc[X["date_recorded"].between(874.50,981.50),"date_recorded_binned"] = "bin1"
    X.loc[X["date_recorded"].between(981.50,1006.50),"date_recorded_binned"] = "bin2"
    X.loc[X["date_recorded"].between(1006.50,np.inf),"date_recorded_binned"] = "bin3"
    X["date_recorded_binned"] = X["date_recorded_binned"].astype("category")
    #Marcar VALORES sin sentido como nan para ser imputados------------------------------------------------------------------------  
    X.loc[X.funder=="0","funder"] = np.nan
    X.loc[X.installer=="0","installer"] = np.nan
    X.loc[X.installer=="-","installer"] = np.nan
    #Drop list ------------------------------------------------------------------------ 
   
    drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by","construction_year"]

    #el drop list 0.8170 más "construction_year"

    #Clusters de latitud y longitud------------------------------------------------------------------------   
    X.drop(drop_list,axis=1,inplace=True)
    #clustering = KMeans(n_clusters=20)
    #X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
    columnas_NA = ["funder","installer","subvillage","public_meeting","scheme_management","scheme_name"]#permit
    for i in columnas_NA: 
        X[i+"isNA"] = X[i].copy()
        X[i+"isNA"]= X[i+"isNA"].isna().astype(int)
    return X

In [ ]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")

it_imp2 = KNNImputer()

for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)



X = preprocess(X)
X_test = preprocess(X_test)
# no entra en preprocessing para poder hacerle al test solo predict
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")




preprocessing = ColumnTransformer([
    ('ordinal_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


#X = it_imp2.fit_transform(X.drop("remainder__id",axis=1))
X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
y_pred = rf.predict(X_test.drop("remainder__id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/KneighborsImputerYMuchoPreproceso.csv",index=None,sep=",")

# Y si en vez de hacerle drop a construction year lo convierto en binaria: 0, no 0

## 0.8162

In [ ]:
def preprocess(X):
    for i in X.select_dtypes(include = "O").columns:
        X[i] = X[i].astype("category")

    #Fechas------------------------------------------------------------------------ 
    X.date_recorded = X.date_recorded.astype("O")
    X.date_recorded = pd.to_datetime(X.date_recorded)
    X['date_recorded'] = (X.date_recorded.max() - X.date_recorded).dt.days
    X.district_code = X.district_code.astype("category")
    X.region_code = X.region_code.astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------   
    X["amount_tsh_binned"] = X["amount_tsh"].copy()
    X.loc[X["amount_tsh_binned"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
    X.loc[X["amount_tsh_binned"].between(1.5,np.inf),"amount_tsh_binned"] = 1
    X["amount_tsh_binned"] = X["amount_tsh_binned"].astype("category")
    #gps_heightoptbinning------------------------------------------------------------------------  
    X.loc[X["gps_height"]<0,"gps_height"] = 0
    X["gps_height_binned"] = X["gps_height"].copy()
    X.loc[X["gps_height"].between(-np.inf,508),"gps_height_binned"] = "bin0"
    X.loc[X["gps_height"].between(508,850.5),"gps_height_binned"] = "bin1"
    X.loc[X["gps_height"].between(850.5,1359.5),"gps_height_binned"] = "bin2"
    X.loc[X["gps_height"].between(1359.5,1567.50),"gps_height_binned"] = "bin3"
    X.loc[X["gps_height"].between(1567.50,1688.50),"gps_height_binned"] = "bin4"
    X.loc[X["gps_height"].between(1688.50,np.inf),"gps_height_binned"] = "bin5"
    X["gps_height_binned"] = X["gps_height_binned"].astype("category")
    #population_binned optbinning------------------------------------------------------------------------ 
    X["population_binned"] = X["population"].copy()
    X.loc[X["population"].between(-np.inf,1.5),"population_binned"] = "bin0"
    X.loc[X["population"].between(1.5,np.inf),"population_binned"] = "bin1"
    X["population_binned"] = X["population_binned"].astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------  
    X["date_recorded_binned"] = X["date_recorded"].copy()
    X.loc[X["date_recorded"].between(-np.inf,874.50),"date_recorded_binned"] = "bin0"
    X.loc[X["date_recorded"].between(874.50,981.50),"date_recorded_binned"] = "bin1"
    X.loc[X["date_recorded"].between(981.50,1006.50),"date_recorded_binned"] = "bin2"
    X.loc[X["date_recorded"].between(1006.50,np.inf),"date_recorded_binned"] = "bin3"
    X["date_recorded_binned"] = X["date_recorded_binned"].astype("category")
    #Marcar VALORES sin sentido como nan para ser imputados------------------------------------------------------------------------  
    X.loc[X.funder=="0","funder"] = np.nan
    X.loc[X.installer=="0","installer"] = np.nan
    X.loc[X.installer=="-","installer"] = np.nan
    #Construction year------------------------------------------------------------------------  
    X["construction_year_iscero"] = X["construction_year"].copy()
    X.loc[X["construction_year"]==0,"construction_year_iscero"] = 1
    X.loc[X["construction_year"]!=0,"construction_year_iscero"] = 0
    #Drop list ------------------------------------------------------------------------ 
   
    drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by","construction_year"]

    #el drop list 0.8170 más "construction_year"

    #Clusters de latitud y longitud------------------------------------------------------------------------   
    X.drop(drop_list,axis=1,inplace=True)
    #clustering = KMeans(n_clusters=20)
    #X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
    columnas_NA = ["funder","installer","subvillage","public_meeting","scheme_management","scheme_name"]#permit
    for i in columnas_NA: 
        X[i+"isNA"] = X[i].copy()
        X[i+"isNA"]= X[i+"isNA"].isna().astype(int)
    return X

In [ ]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")

it_imp2 = KNNImputer()

for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)



X = preprocess(X)
X_test = preprocess(X_test)
# no entra en preprocessing para poder hacerle al test solo predict
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")




preprocessing = ColumnTransformer([
    ('ordinal_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


#X = it_imp2.fit_transform(X.drop("remainder__id",axis=1))
X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
y_pred = rf.predict(X_test.drop("remainder__id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/KneighborsImputerYMuchoPreprocesoMasConstructionYearBinario.csv",index=None,sep=",")

# dejando construction year cero como missing. Próxima vez incluir columna en lista de hacer columnas de NA

## 0.8176

In [ ]:
def preprocess(X):
    for i in X.select_dtypes(include = "O").columns:
        X[i] = X[i].astype("category")

    #Fechas------------------------------------------------------------------------ 
    X.date_recorded = X.date_recorded.astype("O")
    X.date_recorded = pd.to_datetime(X.date_recorded)
    X['date_recorded'] = (X.date_recorded.max() - X.date_recorded).dt.days
    X.district_code = X.district_code.astype("category")
    X.region_code = X.region_code.astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------   
    X["amount_tsh_binned"] = X["amount_tsh"].copy()
    X.loc[X["amount_tsh_binned"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
    X.loc[X["amount_tsh_binned"].between(1.5,np.inf),"amount_tsh_binned"] = 1
    X["amount_tsh_binned"] = X["amount_tsh_binned"].astype("category")
    #gps_heightoptbinning------------------------------------------------------------------------  
    X.loc[X["gps_height"]<0,"gps_height"] = 0
    X["gps_height_binned"] = X["gps_height"].copy()
    X.loc[X["gps_height"].between(-np.inf,508),"gps_height_binned"] = "bin0"
    X.loc[X["gps_height"].between(508,850.5),"gps_height_binned"] = "bin1"
    X.loc[X["gps_height"].between(850.5,1359.5),"gps_height_binned"] = "bin2"
    X.loc[X["gps_height"].between(1359.5,1567.50),"gps_height_binned"] = "bin3"
    X.loc[X["gps_height"].between(1567.50,1688.50),"gps_height_binned"] = "bin4"
    X.loc[X["gps_height"].between(1688.50,np.inf),"gps_height_binned"] = "bin5"
    X["gps_height_binned"] = X["gps_height_binned"].astype("category")
    #population_binned optbinning------------------------------------------------------------------------ 
    X["population_binned"] = X["population"].copy()
    X.loc[X["population"].between(-np.inf,1.5),"population_binned"] = "bin0"
    X.loc[X["population"].between(1.5,np.inf),"population_binned"] = "bin1"
    X["population_binned"] = X["population_binned"].astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------  
    X["date_recorded_binned"] = X["date_recorded"].copy()
    X.loc[X["date_recorded"].between(-np.inf,874.50),"date_recorded_binned"] = "bin0"
    X.loc[X["date_recorded"].between(874.50,981.50),"date_recorded_binned"] = "bin1"
    X.loc[X["date_recorded"].between(981.50,1006.50),"date_recorded_binned"] = "bin2"
    X.loc[X["date_recorded"].between(1006.50,np.inf),"date_recorded_binned"] = "bin3"
    X["date_recorded_binned"] = X["date_recorded_binned"].astype("category")
    #Marcar VALORES sin sentido como nan para ser imputados------------------------------------------------------------------------  
    X["construction_year"] = X.construction_year.max()-X.construction_year
    X.loc[X["construction_year"] ==2013,"construction_year"] = np.nan # si da 2013 la resta fue max - cero
    
    X.loc[X.funder=="0","funder"] = np.nan
    X.loc[X.installer=="0","installer"] = np.nan
    X.loc[X.installer=="-","installer"] = np.nan
    #Drop list ------------------------------------------------------------------------ 
   
    drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]

    #el drop list 0.8170 más "construction_year"

    #Clusters de latitud y longitud------------------------------------------------------------------------   
    X.drop(drop_list,axis=1,inplace=True)
    #clustering = KMeans(n_clusters=20)
    #X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
    columnas_NA = ["funder","installer","subvillage","public_meeting","scheme_management","scheme_name"]#permit
    for i in columnas_NA: 
        X[i+"isNA"] = X[i].copy()
        X[i+"isNA"]= X[i+"isNA"].isna().astype(int)
    return X

In [ ]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from xgboost import cv

In [ ]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")

it_imp2 = KNNImputer()

for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)



X = preprocess(X)
X_test = preprocess(X_test)
# no entra en preprocessing para poder hacerle al test solo predict
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")




preprocessing = ColumnTransformer([
    ('ordinal_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


#X = it_imp2.fit_transform(X.drop("remainder__id",axis=1))
X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
y_pred = rf.predict(X_test.drop("remainder__id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/KneighborsImputerYMuchoPreprocesoMasConstructionYearCeroNAN.csv",index=None,sep=",")

# El de arriba, pero utilizando el encoder que mi función me diga

#### 0.7071

In [ ]:
def preprocess(X):
    for i in X.select_dtypes(include = "O").columns:
        X[i] = X[i].astype("category")

    #Fechas------------------------------------------------------------------------ 
    X.date_recorded = X.date_recorded.astype("O")
    X.date_recorded = pd.to_datetime(X.date_recorded)
    X['date_recorded'] = (X.date_recorded.max() - X.date_recorded).dt.days
    X.district_code = X.district_code.astype("category")
    X.region_code = X.region_code.astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------   
    X["amount_tsh_binned"] = X["amount_tsh"].copy()
    X.loc[X["amount_tsh_binned"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
    X.loc[X["amount_tsh_binned"].between(1.5,np.inf),"amount_tsh_binned"] = 1
    X["amount_tsh_binned"] = X["amount_tsh_binned"].astype("category")
    #gps_heightoptbinning------------------------------------------------------------------------  
    X.loc[X["gps_height"]<0,"gps_height"] = 0
    X["gps_height_binned"] = X["gps_height"].copy()
    X.loc[X["gps_height"].between(-np.inf,508),"gps_height_binned"] = "bin0"
    X.loc[X["gps_height"].between(508,850.5),"gps_height_binned"] = "bin1"
    X.loc[X["gps_height"].between(850.5,1359.5),"gps_height_binned"] = "bin2"
    X.loc[X["gps_height"].between(1359.5,1567.50),"gps_height_binned"] = "bin3"
    X.loc[X["gps_height"].between(1567.50,1688.50),"gps_height_binned"] = "bin4"
    X.loc[X["gps_height"].between(1688.50,np.inf),"gps_height_binned"] = "bin5"
    X["gps_height_binned"] = X["gps_height_binned"].astype("category")
    #population_binned optbinning------------------------------------------------------------------------ 
    X["population_binned"] = X["population"].copy()
    X.loc[X["population"].between(-np.inf,1.5),"population_binned"] = "bin0"
    X.loc[X["population"].between(1.5,np.inf),"population_binned"] = "bin1"
    X["population_binned"] = X["population_binned"].astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------  
    X["date_recorded_binned"] = X["date_recorded"].copy()
    X.loc[X["date_recorded"].between(-np.inf,874.50),"date_recorded_binned"] = "bin0"
    X.loc[X["date_recorded"].between(874.50,981.50),"date_recorded_binned"] = "bin1"
    X.loc[X["date_recorded"].between(981.50,1006.50),"date_recorded_binned"] = "bin2"
    X.loc[X["date_recorded"].between(1006.50,np.inf),"date_recorded_binned"] = "bin3"
    X["date_recorded_binned"] = X["date_recorded_binned"].astype("category")
    #Marcar VALORES sin sentido como nan para ser imputados------------------------------------------------------------------------  
    X["construction_year"] = X.construction_year.max()-X.construction_year
    X.loc[X["construction_year"] ==2013,"construction_year"] = np.nan # si da 2013 la resta fue max - cero
    
    X.loc[X.funder=="0","funder"] = np.nan
    X.loc[X.installer=="0","installer"] = np.nan
    X.loc[X.installer=="-","installer"] = np.nan
    #Drop list ------------------------------------------------------------------------ 
   
    drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]

    #el drop list 0.8170 más "construction_year"

    #Clusters de latitud y longitud------------------------------------------------------------------------   
    X.drop(drop_list,axis=1,inplace=True)
    #clustering = KMeans(n_clusters=20)
    #X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
    columnas_NA = ["funder","installer","subvillage","public_meeting","scheme_management","scheme_name","construction_year"]#permit
    for i in columnas_NA: 
        X[i+"isNA"] = X[i].copy()
        X[i+"isNA"]= X[i+"isNA"].isna().astype(int)
        X[i+"isNA"] = X[i+"isNA"].astype("category")
    return X

In [ ]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")
df = pd.merge(X,y,on="id")

it_imp2 = KNNImputer()

for i in X.select_dtypes(include = "O").columns:
    X[i] = df[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)



X = preprocess(X)
X_test = preprocess(X_test)
# no entra en preprocessing para poder hacerle al test solo predict
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())
#----------------------------------------------------------------------------------------------------------------------
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")


OrdinalEncoder_cols = ['region', 'region_code', 'district_code', 'extraction_type_group',
                        'extraction_type_class', 'payment', 'quality_group', 'quantity_group',
                          'source_type', 'source_class', 'waterpoint_type_group', 'amount_tsh_binned',
                            'gps_height_binned', 'population_binned', 'date_recorded_binned', 
                            'funderisNA', 'installerisNA', 'subvillageisNA', 'public_meetingisNA',
                              'scheme_managementisNA', 'scheme_nameisNA', 'construction_yearisNA',
                                'public_meeting']

OnehotEnc_cols = ['basin', 'waterpoint_type']


TargetEnc_cols = ['scheme_management', 'payment_type', 'water_quality']


FrequencyEnc_cols = ['installer', 'source']

Ordinal_ordered_Encoder_cols = ['funder', 'wpt_name', 'subvillage', 'lga', 'ward', 'scheme_name', 'extraction_type', 'management', 'quantity']


preprocessing = ColumnTransformer([
        ('ordinal', OrdinalEncoder(handle_unknown="use_encoded_value",
                                    unknown_value=10000000,
                                    encoded_missing_value =np.nan,
                                    min_frequency = 0.01), OrdinalEncoder_cols),
        ("ordinal_ordered", feat_eng_OrdinalCategoricalEncoder(encoding_method='ordered',
                                                               variables=None,
                                                               missing_values='ignore'),Ordinal_ordered_Encoder_cols),
        ('onehot', OneHotEncoder(max_categories=7,# Si existen más de 6 valores únicos crea categoría "infrequent"
                                    handle_unknown ="infrequent_if_exist",
                                    drop="first",sparse_output=False), OnehotEnc_cols),

        ('target', TargetEncoder(target_type="multiclass",
                                    cv=2,), TargetEnc_cols),  # default cv 5 me tardaría una eternidad
                                    
        ('Frequency', CountEncoder(handle_unknown="value",
                                    handle_missing="return_nan"), FrequencyEnc_cols),                             
        ('scaler', StandardScaler(), num_cols),  
    ],
    remainder = "passthrough",
    )

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


#X = it_imp2.fit_transform(X.drop("remainder__id",axis=1))
X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
y_pred = rf.predict(X_test.drop("remainder__id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/RF_KNN_FEATURES_FUNCION.csv",index=None,sep=",")

# El mejor, implementación más preprocesado

## 0.8178

In [ ]:
def preprocess(X):
    X.replace('unknown', np.nan)
    X.replace('Unknown', np.nan)
    for i in X.select_dtypes(include = "O").columns:
        X[i] = X[i].astype("category")

    #Fechas------------------------------------------------------------------------ 
    X.date_recorded = X.date_recorded.astype("O")
    X.date_recorded = pd.to_datetime(X.date_recorded)
    X['date_recorded'] = (X.date_recorded.max() - X.date_recorded).dt.days
    X.district_code = X.district_code.astype("category")
    X.region_code = X.region_code.astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------   
    X["amount_tsh_binned"] = X["amount_tsh"].copy()
    X.loc[X["amount_tsh_binned"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
    X.loc[X["amount_tsh_binned"].between(1.5,np.inf),"amount_tsh_binned"] = 1
    X["amount_tsh_binned"] = X["amount_tsh_binned"].astype("category")
    #gps_heightoptbinning------------------------------------------------------------------------  
    X.loc[X["gps_height"]<0,"gps_height"] = 0
    X["gps_height_binned"] = X["gps_height"].copy()
    X.loc[X["gps_height"].between(-np.inf,508),"gps_height_binned"] = "bin0"
    X.loc[X["gps_height"].between(508,850.5),"gps_height_binned"] = "bin1"
    X.loc[X["gps_height"].between(850.5,1359.5),"gps_height_binned"] = "bin2"
    X.loc[X["gps_height"].between(1359.5,1567.50),"gps_height_binned"] = "bin3"
    X.loc[X["gps_height"].between(1567.50,1688.50),"gps_height_binned"] = "bin4"
    X.loc[X["gps_height"].between(1688.50,np.inf),"gps_height_binned"] = "bin5"
    X["gps_height_binned"] = X["gps_height_binned"].astype("category")
    #population_binned optbinning------------------------------------------------------------------------ 
    X["population_binned"] = X["population"].copy()
    X.loc[X["population"].between(-np.inf,1.5),"population_binned"] = "bin0"
    X.loc[X["population"].between(1.5,np.inf),"population_binned"] = "bin1"
    X["population_binned"] = X["population_binned"].astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------  
    X["date_recorded_binned"] = X["date_recorded"].copy()
    X.loc[X["date_recorded"].between(-np.inf,874.50),"date_recorded_binned"] = "bin0"
    X.loc[X["date_recorded"].between(874.50,981.50),"date_recorded_binned"] = "bin1"
    X.loc[X["date_recorded"].between(981.50,1006.50),"date_recorded_binned"] = "bin2"
    X.loc[X["date_recorded"].between(1006.50,np.inf),"date_recorded_binned"] = "bin3"
    X["date_recorded_binned"] = X["date_recorded_binned"].astype("category")
    #Marcar VALORES sin sentido como nan para ser imputados------------------------------------------------------------------------  
    X["construction_year"] = X.construction_year.max()-X.construction_year
    X.loc[X["construction_year"] ==2013,"construction_year"] = np.nan # si da 2013 la resta fue max - cero
    
    X.loc[X.funder=="0","funder"] = np.nan
    X.loc[X.installer=="0","installer"] = np.nan
    X.loc[X.installer=="-","installer"] = np.nan
    #Drop list ------------------------------------------------------------------------ 
   
    drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]

    #el drop list 0.8170 más "construction_year"

    #Clusters de latitud y longitud------------------------------------------------------------------------   
    X.drop(drop_list,axis=1,inplace=True)
    #clustering = KMeans(n_clusters=20)
    #X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
    columnas_NA = ["funder","installer","subvillage","public_meeting","scheme_management","scheme_name","construction_year"]#permit
    for i in columnas_NA: 
        X[i+"isNA"] = X[i].copy()
        X[i+"isNA"]= X[i+"isNA"].isna().astype(int)
        X[i+"isNA"] = X[i+"isNA"].astype("category")
    return X

In [ ]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")


it_imp2 = KNNImputer()

for i in X.select_dtypes(include = "O").columns:
    X[i] = X[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)



X = preprocess(X)
X_test = preprocess(X_test)
# no entra en preprocessing para poder hacerle al test solo predict
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")




preprocessing = ColumnTransformer([
    ('ordinal_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


#X = it_imp2.fit_transform(X.drop("remainder__id",axis=1))
X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
y_pred = rf.predict(X_test.drop("remainder__id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/ReemplazoUnknownPorNaN.csv",index=None,sep=",")

KeyboardInterrupt: 

# A ver qué pasa si reemplazo todos los valores "other - x" por "other"

#### 0.8156

In [ ]:
def preprocess(X):
        #X.basin = X.basin.apply(lambda x: x.split(" ")[0] if x.split(" ")[0] == "Lake" else x)
    for i in ['extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type', 'source',
       'source_type', 'waterpoint_type', 'waterpoint_type_group']:
        
        X[i] = X[i].apply(lambda x: 'other' if isinstance(x, str) and x.split(" ")[0]=="other" else x)

    X.replace('unknown', np.nan)
    X.replace('Unknown', np.nan)
    for i in X.select_dtypes(include = "O").columns:
        X[i] = X[i].astype("category")

    #Fechas------------------------------------------------------------------------ 
    X.date_recorded = X.date_recorded.astype("O")
    X.date_recorded = pd.to_datetime(X.date_recorded)
    X['date_recorded'] = (X.date_recorded.max() - X.date_recorded).dt.days
    X.district_code = X.district_code.astype("category")
    X.region_code = X.region_code.astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------   
    X["amount_tsh_binned"] = X["amount_tsh"].copy()
    X.loc[X["amount_tsh_binned"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
    X.loc[X["amount_tsh_binned"].between(1.5,np.inf),"amount_tsh_binned"] = 1
    X["amount_tsh_binned"] = X["amount_tsh_binned"].astype("category")
    #gps_heightoptbinning------------------------------------------------------------------------  
    X.loc[X["gps_height"]<0,"gps_height"] = 0
    X["gps_height_binned"] = X["gps_height"].copy()
    X.loc[X["gps_height"].between(-np.inf,508),"gps_height_binned"] = "bin0"
    X.loc[X["gps_height"].between(508,850.5),"gps_height_binned"] = "bin1"
    X.loc[X["gps_height"].between(850.5,1359.5),"gps_height_binned"] = "bin2"
    X.loc[X["gps_height"].between(1359.5,1567.50),"gps_height_binned"] = "bin3"
    X.loc[X["gps_height"].between(1567.50,1688.50),"gps_height_binned"] = "bin4"
    X.loc[X["gps_height"].between(1688.50,np.inf),"gps_height_binned"] = "bin5"
    X["gps_height_binned"] = X["gps_height_binned"].astype("category")
    #population_binned optbinning------------------------------------------------------------------------ 
    X["population_binned"] = X["population"].copy()
    X.loc[X["population"].between(-np.inf,1.5),"population_binned"] = "bin0"
    X.loc[X["population"].between(1.5,np.inf),"population_binned"] = "bin1"
    X["population_binned"] = X["population_binned"].astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------  
    X["date_recorded_binned"] = X["date_recorded"].copy()
    X.loc[X["date_recorded"].between(-np.inf,874.50),"date_recorded_binned"] = "bin0"
    X.loc[X["date_recorded"].between(874.50,981.50),"date_recorded_binned"] = "bin1"
    X.loc[X["date_recorded"].between(981.50,1006.50),"date_recorded_binned"] = "bin2"
    X.loc[X["date_recorded"].between(1006.50,np.inf),"date_recorded_binned"] = "bin3"
    X["date_recorded_binned"] = X["date_recorded_binned"].astype("category")
    #Marcar VALORES sin sentido como nan para ser imputados------------------------------------------------------------------------  
    X["construction_year"] = X.construction_year.max()-X.construction_year
    X.loc[X["construction_year"] ==2013,"construction_year"] = np.nan # si da 2013 la resta fue max - cero
    
    X.loc[X.funder=="0","funder"] = np.nan
    X.loc[X.installer=="0","installer"] = np.nan
    X.loc[X.installer=="-","installer"] = np.nan
    #Drop list ------------------------------------------------------------------------ 
   
    drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]

    #el drop list 0.8170 más "construction_year"

    #Clusters de latitud y longitud------------------------------------------------------------------------   
    X.drop(drop_list,axis=1,inplace=True)
    #clustering = KMeans(n_clusters=20)
    #X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
    columnas_NA = ["funder","installer","subvillage","public_meeting","scheme_management","scheme_name","construction_year"]#permit
    for i in columnas_NA: 
        X[i+"isNA"] = X[i].copy()
        X[i+"isNA"]= X[i+"isNA"].isna().astype(int)
        X[i+"isNA"] = X[i+"isNA"].astype("category")
    return X

In [ ]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")


it_imp2 = KNNImputer()

for i in X.select_dtypes(include = "O").columns:
    X[i] = X[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)



X = preprocess(X)
X_test = preprocess(X_test)
# no entra en preprocessing para poder hacerle al test solo predict
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")




preprocessing = ColumnTransformer([
    ('ordinal_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


#X = it_imp2.fit_transform(X.drop("remainder__id",axis=1))
X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
y_pred = rf.predict(X_test.drop("remainder__id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/ReemplazoOtherXporOOther.csv",index=None,sep=",")

# Unas cuantas erratas:

###  añadirle min freq al ordinal encoder!! 
### scheme management tiene una a la que mosca con hacerle eso del min freq
### X.replace('unknown', np.nan) necesita inplace o asignación
### no se actualizó la lista de Nans a la que hacerles columnas propias 
### además de unknown hay columnas con "none"
### hacerle lower a unas cuantas columnas
### hacer algo con las categorías en train que no salen en test

## 0.8136

In [ ]:
def preprocess_train_test(X,X_test = None):
    """
    pasa X_test sólo al preprocesar X
    """
        #X.basin = X.basin.apply(lambda x: x.split(" ")[0] if x.split(" ")[0] == "Lake" else x)
    cols_mas_unique_en_train = ["subvillage","installer","funder","scheme_name","wpt_name"]

    X = X.replace('unknown', np.nan)
    X = X.replace('Unknown', np.nan)
    X = X.replace('none', np.nan)
    cols_pasar_a_lower = ["installer","scheme_name","wpt_name"]
    for col in cols_pasar_a_lower:
        X[col] = X[col].apply(lambda x: str(x).lower())

    if X_test is not None:
        print("si")
        cols_mas_unique_en_train = ["subvillage","installer","funder","scheme_name","wpt_name",
                                    "region_code","ward","extraction_type"]
        for i in cols_mas_unique_en_train:
            unicos_en_test = X_test[i].unique().tolist()
            X[i] = [x if x in unicos_en_test else np.nan for x in X[i].values]
            #X[i] = [x if x in unicos_en_test else x.split(" ")[0] for x in X[i].values]

    for i in X.select_dtypes(include = "O").columns:
        X[i] = X[i].astype("category")

    #Fechas------------------------------------------------------------------------ 
    X.date_recorded = X.date_recorded.astype("O")
    X.date_recorded = pd.to_datetime(X.date_recorded)
    X['date_recorded'] = (X.date_recorded.max() - X.date_recorded).dt.days
    #------------------------------------------------------------------------------------------------------------------------------------------
    X.district_code = X.district_code.astype("category")
    X.region_code = X.region_code.astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------   
    X["amount_tsh_binned"] = X["amount_tsh"].copy()
    X.loc[X["amount_tsh_binned"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
    X.loc[X["amount_tsh_binned"].between(1.5,np.inf),"amount_tsh_binned"] = 1
    X["amount_tsh_binned"] = X["amount_tsh_binned"].astype("category")
    #gps_heightoptbinning------------------------------------------------------------------------  
    X.loc[X["gps_height"]<0,"gps_height"] = 0
    X["gps_height_binned"] = X["gps_height"].copy()
    X.loc[X["gps_height"].between(-np.inf,508),"gps_height_binned"] = "bin0"
    X.loc[X["gps_height"].between(508,850.5),"gps_height_binned"] = "bin1"
    X.loc[X["gps_height"].between(850.5,1359.5),"gps_height_binned"] = "bin2"
    X.loc[X["gps_height"].between(1359.5,1567.50),"gps_height_binned"] = "bin3"
    X.loc[X["gps_height"].between(1567.50,1688.50),"gps_height_binned"] = "bin4"
    X.loc[X["gps_height"].between(1688.50,np.inf),"gps_height_binned"] = "bin5"
    X["gps_height_binned"] = X["gps_height_binned"].astype("category")
    #population_binned optbinning------------------------------------------------------------------------ 
    X["population_binned"] = X["population"].copy()
    X.loc[X["population"].between(-np.inf,1.5),"population_binned"] = "bin0"
    X.loc[X["population"].between(1.5,np.inf),"population_binned"] = "bin1"
    X["population_binned"] = X["population_binned"].astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------  
    X["date_recorded_binned"] = X["date_recorded"].copy()
    X.loc[X["date_recorded"].between(-np.inf,874.50),"date_recorded_binned"] = "bin0"
    X.loc[X["date_recorded"].between(874.50,981.50),"date_recorded_binned"] = "bin1"
    X.loc[X["date_recorded"].between(981.50,1006.50),"date_recorded_binned"] = "bin2"
    X.loc[X["date_recorded"].between(1006.50,np.inf),"date_recorded_binned"] = "bin3"
    X["date_recorded_binned"] = X["date_recorded_binned"].astype("category")
    #Marcar VALORES sin sentido como nan para ser imputados------------------------------------------------------------------------  
    X["construction_year"] = X.construction_year.max()-X.construction_year
    X.loc[X["construction_year"] ==2013,"construction_year"] = np.nan # si da 2013 la resta fue max - cero
    
    X.loc[X.funder=="0","funder"] = np.nan
    X.loc[X.installer=="0","installer"] = np.nan
    X.loc[X.installer=="-","installer"] = np.nan
    #Drop list ------------------------------------------------------------------------ 
    drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]   
    X.drop(drop_list,axis=1,inplace=True)
    #Clusters de latitud y longitud, se hace fuera para que X_test sea transform sin fit------------------------------------------------------------------------
    #clustering = KMeans(n_clusters=20)
    #X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
    
    columnas_NA = X.columns[X.isna().any()].tolist()
    columnas_NA = [x for x in columnas_NA if x not in ['scheme_name', 'wpt_name', 'ward', 'extraction_type', 'region_code']]# Nas por sobra de categorías, sin nulls en X_test
    for i in columnas_NA: 
        X[i+"isNA"] = X[i].copy()
        X[i+"isNA"]= X[i+"isNA"].isna().astype(int)
        X[i+"isNA"] = X[i+"isNA"].astype("category")
    return X

In [ ]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")


it_imp2 = KNNImputer()

for i in X.select_dtypes(include = "O").columns:
    X[i] = X[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)


X_test = preprocess_train_test(X_test)
X = preprocess_train_test(X,X_test)

# no entra en preprocessing para poder hacerle al test solo predict
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")




preprocessing = ColumnTransformer([
    ('ordinal_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan,
                                  min_frequency = 0.01), cat_cols),  # si menos de 1% de los datos
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


#X = it_imp2.fit_transform(X.drop("remainder__id",axis=1))
X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
y_pred = rf.predict(X_test.drop("remainder__id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/ArregloErratas.csv",index=None,sep=",")

si


## Ahora cambio valores no en X_test a constante "no_en_test" en vez de a NAN

### 0.8135

In [ ]:
def preprocess_train_test(X,X_test = None):
    """
    pasa X_test sólo al preprocesar X
    """
        #X.basin = X.basin.apply(lambda x: x.split(" ")[0] if x.split(" ")[0] == "Lake" else x)
    cols_mas_unique_en_train = ["subvillage","installer","funder","scheme_name","wpt_name"]

    X = X.replace('unknown', np.nan)# se pasan a nan, pero son muy significativos, por lo que me aseguro de crear columna de que es NAN imputado
    X = X.replace('Unknown', np.nan)
    X = X.replace('none', np.nan)
    
    cols_pasar_a_lower = ["installer","scheme_name","wpt_name"]
    for col in cols_pasar_a_lower:
        X[col] = X[col].apply(lambda x: str(x).lower())
    
    """
    """
    if X_test is not None:
        print("si")
        cols_mas_unique_en_train = ["subvillage","installer","funder","scheme_name","wpt_name","ward","extraction_type"]#"region_code"
        for i in cols_mas_unique_en_train:
            unicos_en_test = X_test[i].unique().tolist()
            X[i] = [x if x in unicos_en_test else "no_en_test" for x in X[i].values]
            #X[i] = [x if x in unicos_en_test else x.split(" ")[0] for x in X[i].values]
        
    

    for i in X.select_dtypes(include = "O").columns:
        X[i] = X[i].astype("category")

    #Fechas------------------------------------------------------------------------ 
    X.date_recorded = X.date_recorded.astype("O")
    X.date_recorded = pd.to_datetime(X.date_recorded)
    X['date_recorded'] = (X.date_recorded.max() - X.date_recorded).dt.days
    #------------------------------------------------------------------------------------------------------------------------------------------
    X.district_code = X.district_code.astype("category")
    X.region_code = X.region_code.astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------   
    X["amount_tsh_binned"] = X["amount_tsh"].copy()
    X.loc[X["amount_tsh_binned"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
    X.loc[X["amount_tsh_binned"].between(1.5,np.inf),"amount_tsh_binned"] = 1
    X["amount_tsh_binned"] = X["amount_tsh_binned"].astype("category")
    #gps_heightoptbinning------------------------------------------------------------------------  
    X.loc[X["gps_height"]<0,"gps_height"] = 0
    X["gps_height_binned"] = X["gps_height"].copy()
    X.loc[X["gps_height"].between(-np.inf,508),"gps_height_binned"] = "bin0"
    X.loc[X["gps_height"].between(508,850.5),"gps_height_binned"] = "bin1"
    X.loc[X["gps_height"].between(850.5,1359.5),"gps_height_binned"] = "bin2"
    X.loc[X["gps_height"].between(1359.5,1567.50),"gps_height_binned"] = "bin3"
    X.loc[X["gps_height"].between(1567.50,1688.50),"gps_height_binned"] = "bin4"
    X.loc[X["gps_height"].between(1688.50,np.inf),"gps_height_binned"] = "bin5"
    X["gps_height_binned"] = X["gps_height_binned"].astype("category")
    #population_binned optbinning------------------------------------------------------------------------ 
    X["population_binned"] = X["population"].copy()
    X.loc[X["population"].between(-np.inf,1.5),"population_binned"] = "bin0"
    X.loc[X["population"].between(1.5,np.inf),"population_binned"] = "bin1"
    X["population_binned"] = X["population_binned"].astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------  
    X["date_recorded_binned"] = X["date_recorded"].copy()
    X.loc[X["date_recorded"].between(-np.inf,874.50),"date_recorded_binned"] = "bin0"
    X.loc[X["date_recorded"].between(874.50,981.50),"date_recorded_binned"] = "bin1"
    X.loc[X["date_recorded"].between(981.50,1006.50),"date_recorded_binned"] = "bin2"
    X.loc[X["date_recorded"].between(1006.50,np.inf),"date_recorded_binned"] = "bin3"
    X["date_recorded_binned"] = X["date_recorded_binned"].astype("category")
    #Marcar VALORES sin sentido como nan para ser imputados------------------------------------------------------------------------  
    X["construction_year"] = X.construction_year.max()-X.construction_year
    X.loc[X["construction_year"] ==2013,"construction_year"] = np.nan # si da 2013 la resta fue max - cero
    
    #X.loc[X.funder=="0","funder"] = np.nan
    X["funder"] = X["funder"].replace("0",np.nan)
    #X.loc[X.installer=="0","installer"] = np.nan
    #X.loc[X.installer=="-","installer"] = np.nan
    X["installer"] = X["installer"].replace("0",np.nan)
    X["installer"] = X["installer"].replace("-",np.nan)
    #Drop list ------------------------------------------------------------------------ 
    drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]   
    X.drop(drop_list,axis=1,inplace=True)
    #Clusters de latitud y longitud, se hace fuera para que X_test sea transform sin fit------------------------------------------------------------------------
    #clustering = KMeans(n_clusters=20)
    #X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
    
    columnas_NA = X.columns[X.isna().any()].tolist()
    columnas_NA = [x for x in columnas_NA if x not in ['scheme_name', 'wpt_name', 'ward', 'extraction_type', 'region_code']]# Nas por sobra de categorías, sin nulls en X_test
    for i in columnas_NA: 
        X[i+"isNA"] = X[i].copy()
        X[i+"isNA"]= X[i+"isNA"].isna().astype(int)
        X[i+"isNA"] = X[i+"isNA"].astype("category")
    return X

In [ ]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")


it_imp2 = KNNImputer()

for i in X.select_dtypes(include = "O").columns:
    X[i] = X[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)


X_test = preprocess_train_test(X_test)
X = preprocess_train_test(X,X_test)

# no entra en preprocessing para poder hacerle al test solo predict
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())


cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")




preprocessing = ColumnTransformer([
    ('ordinal_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan,
                                  min_frequency = 0.01), cat_cols),  # si menos de 1% de los datos
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())

#X = it_imp2.fit_transform(X.drop("remainder__id",axis=1))
X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
y_pred = rf.predict(X_test.drop("remainder__id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/NoEnXtestAConstante.csv",index=None,sep=",")

si
preprocesado


# QuedandomeConPrimeraParteDeCatsNoEnTest 

#### 0.8128

In [ ]:
def preprocess_train_test(X,X_test = None):
    """
    pasa X_test sólo al preprocesar X
    """
        #X.basin = X.basin.apply(lambda x: x.split(" ")[0] if x.split(" ")[0] == "Lake" else x)
    cols_mas_unique_en_train = ["subvillage","installer","funder","scheme_name","wpt_name"]

    X = X.replace('unknown', np.nan)# se pasan a nan, pero son muy significativos, por lo que me aseguro de crear columna de que es NAN imputado
    X = X.replace('Unknown', np.nan)
    X = X.replace('none', np.nan)
    
    cols_pasar_a_lower = ["installer","scheme_name","wpt_name"]
    for col in cols_pasar_a_lower:
        X[col] = X[col].apply(lambda x: str(x).lower())
    
    
    """
    """ 
    if X_test is not None:
        print("si")
        cols_mas_unique_en_train = ["subvillage","installer","funder","scheme_name","wpt_name","ward","extraction_type"]#"region_code"
        for i in cols_mas_unique_en_train:
            unicos_en_test = X_test[i].unique().tolist()
            #X[i] = [x if x in unicos_en_test else "no_en_test" for x in X[i].values]
            X[i] = [x if x in unicos_en_test else str(x).split(" ")[0] for x in X[i].values]
      
    

    for i in X.select_dtypes(include = "O").columns:
        X[i] = X[i].astype("category")

    #Fechas------------------------------------------------------------------------ 
    X.date_recorded = X.date_recorded.astype("O")
    X.date_recorded = pd.to_datetime(X.date_recorded)
    X['date_recorded'] = (X.date_recorded.max() - X.date_recorded).dt.days
    #------------------------------------------------------------------------------------------------------------------------------------------
    X.district_code = X.district_code.astype("category")
    X.region_code = X.region_code.astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------   
    X["amount_tsh_binned"] = X["amount_tsh"].copy()
    X.loc[X["amount_tsh_binned"].between(-np.inf,1.5),"amount_tsh_binned"] = 0
    X.loc[X["amount_tsh_binned"].between(1.5,np.inf),"amount_tsh_binned"] = 1
    X["amount_tsh_binned"] = X["amount_tsh_binned"].astype("category")
    #gps_heightoptbinning------------------------------------------------------------------------  
    X.loc[X["gps_height"]<0,"gps_height"] = 0
    X["gps_height_binned"] = X["gps_height"].copy()
    X.loc[X["gps_height"].between(-np.inf,508),"gps_height_binned"] = "bin0"
    X.loc[X["gps_height"].between(508,850.5),"gps_height_binned"] = "bin1"
    X.loc[X["gps_height"].between(850.5,1359.5),"gps_height_binned"] = "bin2"
    X.loc[X["gps_height"].between(1359.5,1567.50),"gps_height_binned"] = "bin3"
    X.loc[X["gps_height"].between(1567.50,1688.50),"gps_height_binned"] = "bin4"
    X.loc[X["gps_height"].between(1688.50,np.inf),"gps_height_binned"] = "bin5"
    X["gps_height_binned"] = X["gps_height_binned"].astype("category")
    #population_binned optbinning------------------------------------------------------------------------ 
    X["population_binned"] = X["population"].copy()
    X.loc[X["population"].between(-np.inf,1.5),"population_binned"] = "bin0"
    X.loc[X["population"].between(1.5,np.inf),"population_binned"] = "bin1"
    X["population_binned"] = X["population_binned"].astype("category")
    #Marcar 0 en amount como dijo optbinning------------------------------------------------------------------------  
    X["date_recorded_binned"] = X["date_recorded"].copy()
    X.loc[X["date_recorded"].between(-np.inf,874.50),"date_recorded_binned"] = "bin0"
    X.loc[X["date_recorded"].between(874.50,981.50),"date_recorded_binned"] = "bin1"
    X.loc[X["date_recorded"].between(981.50,1006.50),"date_recorded_binned"] = "bin2"
    X.loc[X["date_recorded"].between(1006.50,np.inf),"date_recorded_binned"] = "bin3"
    X["date_recorded_binned"] = X["date_recorded_binned"].astype("category")
    #Marcar VALORES sin sentido como nan para ser imputados------------------------------------------------------------------------  
    X["construction_year"] = X.construction_year.max()-X.construction_year
    X.loc[X["construction_year"] ==2013,"construction_year"] = np.nan # si da 2013 la resta fue max - cero
    
    #X.loc[X.funder=="0","funder"] = np.nan
    X["funder"] = X["funder"].replace("0",np.nan)
    #X.loc[X.installer=="0","installer"] = np.nan
    #X.loc[X.installer=="-","installer"] = np.nan
    X["installer"] = X["installer"].replace("0",np.nan)
    X["installer"] = X["installer"].replace("-",np.nan)
    #Drop list ------------------------------------------------------------------------ 
    drop_list  = ["management_group","permit","population","amount_tsh","num_private","recorded_by"]   
    X.drop(drop_list,axis=1,inplace=True)
    #Clusters de latitud y longitud, se hace fuera para que X_test sea transform sin fit------------------------------------------------------------------------
    #clustering = KMeans(n_clusters=20)
    #X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
    
    columnas_NA = X.columns[X.isna().any()].tolist()
    columnas_NA = [x for x in columnas_NA if x not in ['scheme_name', 'wpt_name', 'ward', 'extraction_type', 'region_code']]# Nas por sobra de categorías, sin nulls en X_test
    for i in columnas_NA: 
        X[i+"isNA"] = X[i].copy()
        X[i+"isNA"]= X[i+"isNA"].isna().astype(int)
        X[i+"isNA"] = X[i+"isNA"].astype("category")
    return X

In [ ]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")


it_imp2 = KNNImputer()

for i in X.select_dtypes(include = "O").columns:
    X[i] = X[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)


X_test = preprocess_train_test(X_test)
X = preprocess_train_test(X,X_test)

# no entra en preprocessing para poder hacerle al test solo predict
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())


cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")




preprocessing = ColumnTransformer([
    ('ordinal_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan,
                                  min_frequency = 0.01), cat_cols),  # si menos de 1% de los datos
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())

#X = it_imp2.fit_transform(X.drop("remainder__id",axis=1))
X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
y_pred = rf.predict(X_test.drop("remainder__id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/QuedandomeConPrimeraParteDeCatsNoEnTest.csv",index=None,sep=",")

si


## PRUEBO Cambios propuestos por validación cruzada.

## 0.8156

In [12]:
def preprocess(X,drop=False,X_test = None):
    """
    pasa X_test sólo al preprocesar X
    """
    if X_test is not None:
        cols_mas_unique_en_train = ["subvillage","installer","funder","scheme_name","wpt_name",
                                    "region_code","ward","extraction_type"]
        for i in cols_mas_unique_en_train:
            unicos_en_test = X_test[i].unique().tolist()
            X[i] = [x if x in unicos_en_test else np.nan for x in X[i].values]
            #X[i] = [x if x in unicos_en_test else x.split(" ")[0] for x in X[i].values]

    for i in X.select_dtypes(include = "O").columns:
        X[i] = X[i].astype("category")

    #Fechas------------------------------------------------------------------------ 
    X.date_recorded = X.date_recorded.astype("O")
    X.date_recorded = pd.to_datetime(X.date_recorded)
    X['date_recorded'] = (X.date_recorded.max() - X.date_recorded).dt.days

    #population_binned optbinning------------------------------------------------------------------------ 
    X["population_binned"] = X["population"].copy()
    X.loc[X["population"].between(-np.inf,1.5),"population_binned"] = "bin0"
    X.loc[X["population"].between(1.5,np.inf),"population_binned"] = "bin1"
    X["population_binned"] = X["population_binned"].astype("category")
    #Marcar VALORES sin sentido como nan para ser imputados------------------------------------------------------------------------  
    X["construction_year"] = X.construction_year.max()-X.construction_year
    X.loc[X["construction_year"] ==2013,"construction_year"] = np.nan # si da 2013 la resta fue max - cero
    #Drop list ------------------------------------------------------------------------ 

    drop_list  = ["amount_tsh","num_private","recorded_by"]   
    X.drop(drop_list,axis=1,inplace=True)
    #Clusters de latitud y longitud, se hace fuera para que X_test sea transform sin fit------------------------------------------------------------------------
    #clustering = KMeans(n_clusters=20)
    #X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
    
    return X

In [48]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")


it_imp2 = KNNImputer()

for i in X.select_dtypes(include = "O").columns:
    X[i] = X[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)

X = preprocess(X)
X_test = preprocess(X_test)
# no entra en preprocessing para poder hacerle al test solo predict
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")




preprocessing = ColumnTransformer([
    ('ordinal_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan,
                                  min_frequency = 0.005), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


#X = it_imp2.fit_transform(X.drop("remainder__id",axis=1))
X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier(n_jobs=-1)
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
y_pred = rf.predict(X_test.drop("remainder__id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/SacoDropsYPreprocesoCambioMinFreq.csv",index=None,sep=",")

## Intento otra vez con lo que me dijo cross_validate, pero sin min_freq y con hiperparámetros según gridsearch.

## 0.8213

In [8]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")


it_imp2 = KNNImputer()

for i in X.select_dtypes(include = "O").columns:
    X[i] = X[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)

X = preprocess(X)
X_test = preprocess(X_test)
# no entra en preprocessing para poder hacerle al test solo predict
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")




preprocessing = ColumnTransformer([
    ('ordinal_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


#X = it_imp2.fit_transform(X.drop("remainder__id",axis=1))
X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

rf = RandomForestClassifier(bootstrap=False,max_depth=20,max_features="sqrt",min_samples_leaf=2,n_estimators=150,n_jobs=-1)
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
y_pred = rf.predict(X_test.drop("remainder__id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/GridSearchRF.csv",index=None,sep=",")

# Intento otra vez, esta vez con el mejor set de hiperparámetros con bootsrap = True, pues entiendo que esto ayuda al random forest a generalizar mejor.

## Empeora el resultado 0.8190

In [9]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")


it_imp2 = KNNImputer()

for i in X.select_dtypes(include = "O").columns:
    X[i] = X[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)

X = preprocess(X)
X_test = preprocess(X_test)
# no entra en preprocessing para poder hacerle al test solo predict
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")




preprocessing = ColumnTransformer([
    ('ordinal_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


#X = it_imp2.fit_transform(X.drop("remainder__id",axis=1))
X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

#{'bootstrap': True, 'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 150}
rf = RandomForestClassifier(bootstrap=True,max_depth=20,max_features="log2",min_samples_leaf=1,min_samples_split=3,n_estimators=150,n_jobs=-1)
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
y_pred = rf.predict(X_test.drop("remainder__id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/GridSearchBootstrappedRF.csv",index=None,sep=",")

# Intento otra vez, el mejor hasta ahora, arreglando que no había incluido el paso del preprocesamiento que convertía en Nan las categorías que no aparecen en X_test

## Empeoró un poco 0.8208

In [14]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")


it_imp2 = KNNImputer()

for i in X.select_dtypes(include = "O").columns:
    X[i] = X[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)


X_test = preprocess(X_test)
X = preprocess(X,X_test)
# no entra en preprocessing para poder hacerle al test solo predict
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")




preprocessing = ColumnTransformer([
    ('ordinal_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


#X = it_imp2.fit_transform(X.drop("remainder__id",axis=1))
X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

#{'bootstrap': True, 'max_depth': 20, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 150}
rf = RandomForestClassifier(bootstrap=False,max_depth=20,max_features="sqrt",min_samples_leaf=2,n_estimators=150,n_jobs=-1)
rf.fit(X.drop("remainder__id",axis=1),y.status_group.values)
y_pred = rf.predict(X_test.drop("remainder__id",axis=1))

X_test["status_group"] = rf.predict(X_test.drop("remainder__id",axis=1))

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)
X_test.to_csv("Out/ConNANsDeCatsNoEnX_test.csv",index=None,sep=",")

# Modelo de voting monstruoso

## 0.8151 :(

In [17]:
X = pd.read_csv("In/Training_set_values.csv")
y = pd.read_csv("In/Training_set_labels.csv")
X_test = pd.read_csv("In/Test_set_values.csv")


it_imp2 = KNNImputer()

for i in X.select_dtypes(include = "O").columns:
    X[i] = X[i].astype("category")

le= LabelEncoder()
y["status_group"] = le.fit_transform(y.status_group)


X_test = preprocess(X_test)
X = preprocess(X,X_test)
# no entra en preprocessing para poder hacerle al test solo predict
clustering = KMeans(n_clusters=20)
X["location_cluster"] = clustering.fit_predict(X[["latitude","longitude"]].to_numpy())
X_test["location_cluster"] = clustering.predict(X_test[["latitude","longitude"]].to_numpy())
cat_cols = X.select_dtypes(exclude=np.number).columns.tolist()
num_cols = X.select_dtypes(include=np.number).columns.tolist()

num_cols.remove("id")




preprocessing = ColumnTransformer([
    ('ordinal_enc', OrdinalEncoder(handle_unknown="use_encoded_value",
                                  unknown_value=10000000,
                                  encoded_missing_value =np.nan), cat_cols),  
    ('scaler', StandardScaler(), num_cols),  
],
remainder = "passthrough",
)

X = pd.DataFrame(data=preprocessing.fit_transform(X,y=y.status_group.values), columns=preprocessing.get_feature_names_out())
X_test = pd.DataFrame(data=preprocessing.transform(X_test), columns=preprocessing.get_feature_names_out())


#X = it_imp2.fit_transform(X.drop("remainder__id",axis=1))
X = pd.DataFrame(data=it_imp2.fit_transform(X), columns=it_imp2.get_feature_names_out())
#X_test = it_imp2.transform(X)
X_test = pd.DataFrame(data=it_imp2.transform(X_test), columns=it_imp2.get_feature_names_out())

In [41]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

rf = RandomForestClassifier(bootstrap=False, max_depth=20, max_features="sqrt", min_samples_leaf=2, n_estimators=150)
rf_resampled = Pipeline([
    ('smote', SMOTE(sampling_strategy="minority")), 
    ('rf', RandomForestClassifier(bootstrap=False, max_depth=20, max_features="sqrt", min_samples_leaf=2, n_estimators=150))  
])

mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=1000)# un multilayer perceptron, entiendo que es como una mini red neuronal
log_reg = LogisticRegression(solver = "liblinear",max_iter=10000, penalty="l1")# penalty l1 es una regresión lasso, normalmente funcionan mejor en datos con demasiadas columnas
grad_boost = GradientBoostingClassifier()
SVM = SVC(kernel="rbf")# rbf entiendo es el mejor porque es capaz de encontrar patrones en dimensiones infinitas



Clasificador_monstruo = VotingClassifier(estimators=[
      ('random_fores', rf),("rf_smote",rf_resampled), ('multilayer_perceptron', mlp),("logistic",log_reg),
        ('gradient_boosting', grad_boost),("SupportVectorMachine",SVM)],
          weights=[2, 1, 1,1,1,1], voting='hard')

Clasificador_monstruo.fit(X.drop("remainder__id",axis=1),y.status_group.values)

VotingClassifier(estimators=[('random_fores',
                              RandomForestClassifier(bootstrap=False,
                                                     max_depth=20,
                                                     min_samples_leaf=2,
                                                     n_estimators=150)),
                             ('rf_smote',
                              Pipeline(steps=[('smote',
                                               SMOTE(sampling_strategy='minority')),
                                              ('rf',
                                               RandomForestClassifier(bootstrap=False,
                                                                      max_depth=20,
                                                                      min_samples_leaf=2,
                                                                      n_estimators=150))])),
                             ('multilayer_perceptron',
                              MLPClassifier(max_iter=1000)),
                             ('logistic',
                              LogisticRegression(max_iter=10000, penalty='l1',
                                                 solver='liblinear')),
                             ('gradient_boosting',
                              GradientBoostingClassifier()),
                             ('SupportVectorMachine', SVC())],
                 weights=[2, 1, 1, 1, 1, 1])

In [42]:
y_pred = Clasificador_monstruo.predict(X_test.drop("remainder__id",axis=1))

X_test["status_group"] = y_pred

X_test.remainder__id = X_test.remainder__id.astype(int)
X_test.rename({"remainder__id":"id","predictions":"status_group"},axis=1,inplace=True)

X_test = X_test[["id","status_group"]]
X_test.status_group = le.inverse_transform(X_test.status_group)

X_test.to_csv("Out/Clasificador_monstruo.csv",index=None,sep=",")